<a href="https://colab.research.google.com/github/BRIJNANDA1979/BIGEARTHNET/blob/main/Using_pyspark_on_Copy_of_BigEarthNet_CNN_for_uniquely_labelled_46_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BigEarthNet unique labelled data with 46 classes from data uploaded on google drive.

Step  1: to create geotiff image using 12 bands of Sentinel-2 imagery 
Step 2 :  to create batch of data with labels
Step 3: Normalize data
Step 4: Transform data in proper shape
Step 5: Data input to CNN
Step 6: Reslut and prediction of data


In [39]:
import os
import pandas as pd

In [40]:
src_path = '/content/drive/MyDrive/BigEarthNet unique labelled'

In [41]:
!pip install rasterio

In [42]:
!pip install glob2

In [43]:
dirs = os.listdir(src_path)  #list of all classes
print(dirs)
print(len(dirs))

['Agro-forestry areas', 'Airports', 'Annual crops associated with permanent crops', 'Coastal lagoons', 'Complex cultivation patterns', 'Discontinuous urban fabric', 'Dump sites', 'Estuaries', 'Fruit trees and berry plantations', 'Green urban areas', 'Industrial or commercial units', 'Inland marshes', 'Intertidal flats', 'Mineral extraction sites', 'Mixed forest', 'Moors and heathland', 'Natural grassland', 'Permanently irrigated land', 'Rice fields', 'Salt marshes', 'Sclerophyllous vegetation', 'Sparsely vegetated areas', 'Sport and leisure facilities', 'Transitional woodlandshrub', 'Non-irrigated arable land', 'Water bodies', 'Water courses', '.ipynb_checkpoints']
28


In [ ]:
# TESTING the input data..before merging 12 bands to geotiff
%%timeit
import glob2 # glob2 is faster than glob
for dir in dirs:
  sub_dirs = os.listdir(os.path.join(src_path,dir))
  src_folders_path = os.path.join(src_path,dir)
  for sub in sub_dirs:
    path=os.path.join(src_folders_path,sub)
    #print(len(sub_dirs))
    search = '*_B*.tif'   # 12 Band files end with B01,B02....
    files_batch = []
    files_batch = glob2.glob(os.path.join(path,search)) #search all tif band files 
    files_batch =sorted(files_batch)
    #print(files_batch) 
    if len(files_batch) < 12:  # each source tif files folder must have 12 tif files
      print(len(files_batch))
      print(dir)
      exit_parameter  = True
      break
    else:
      continue
    if exit_parameter == True:
      break

In [ ]:
import pandas as pd
src_path='/content/drive/MyDrive/BigEarthNet unique labelled'
classes = os.listdir(src_path)
classes

In [ ]:
# Start PySpark session to read data in spark Dataframe

!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark=SparkSession.builder.appName('BigEarthNet').getOrCreate()
Df = spark.read.csv('/content/drive/MyDrive/Data/Data with Labels.csv',inferSchema=True, header =True)
Df.printSchema()


In [ ]:
Df.show()

In [ ]:
Df.describe()

In [ ]:
# Collect the data to Python List
dataCollect = Df.collect()   #https://sparkbyexamples.com/pyspark/pyspark-loop-iterate-through-rows-in-dataframe/
for row in dataCollect:
    print(row[1])

In [ ]:
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image
from rasterio.plot import adjust_band
import numpy as np
#print(files) #geotiff files

fpath = '/content/drive/MyDrive/BigEarthNet unique labelled'

batch=[]
for row in dataCollect:    # Using Spark dataframes
  f = fpath + '/' + row[1] + '/' + row[0] + '/' + row[0] + '.tif'    
  img_data = rasterio.open(f)
  image = img_data.read()
  image_norm = adjust_band(image) # normalize bands to range between 1.0 to 0.0
  image_reshaped = reshape_as_image(image_norm) # reshape to [rows, cols, bands]
  print(image_reshaped.shape)
  print(img_data.shape)
  batch.append(image_reshaped)
print(len(batch))
batch=np.asarray(batch)
print(batch.shape)
X=batch    # Input for CNN

In [ ]:
print(len(classes))
l=len(classes)
code_classes = list(enumerate(classes,0))   # code the classes
code_classes

In [ ]:
#convert list to array
import numpy as np
batch=np.asarray(batch)
print(batch.shape)
y=dataCollect['code']
print(y)
y=np.array(y)
print(y)

In [ ]:
print(X.shape)    # data is ready for CNN
print(y.shape)


In [ ]:
print(batch[0])  #first tiff image normalized

In [ ]:
!pip install keras.utils


In [ ]:
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)
y_train = np_utils.to_categorical(y_train, len(classes))
y_test = np_utils.to_categorical(y_test, len(classes))

In [ ]:
print(y_train[1681:1683])  # visualize y_train data

In [ ]:
input_shape = (20,20,12)
num_classes = len(classes)

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint

In [ ]:
model = keras.models.Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(
   loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']
)

In [ ]:
from keras.callbacks import EarlyStopping 

In [ ]:
history = model.fit(X_train, y_train, batch_size = 30, epochs = 150, validation_data = (X_test, y_test),
                    callbacks = [EarlyStopping(monitor = 'val_loss', patience = 20)])

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
pred = model.predict(X_test) 
pred = np.argmax(pred, axis = 1)[:100] 
label = np.argmax(y_test,axis = 1)[:100] 

print(pred) 
print(label)

In [ ]:
prediction = model.predict(X_test)
print(prediction.flatten()) 

In [ ]:
# Get the count of accurately predicted records - (1)
print(sum(pred == label))
# Get the count of total records - (2)
print(len(pred))

# Categorical Accuracy = (1)/(2)
CalculatedCategoricalAccuracy = sum(pred == label)/len(pred)
print(CalculatedCategoricalAccuracy)



In [ ]:
# Plot training & validation accuracy values
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()